In [1]:
%config IPCompleter.greedy=True

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [2]:
import import_ipynb
from functions import calc_vpc
from visuals import colors
from visuals import map_position_to_color

importing Jupyter notebook from functions.ipynb
importing Jupyter notebook from visuals.ipynb


In [3]:
import os
import requests
import pandas

# Get parameters
# SEASON = os.environ["FPL_SEASON"]
SEASON = '2018-19'

# Get current gameweek
# URL = "https://fantasy.premierleague.com/api/bootstrap-static"
# DATA = requests.get(URL).json()
# CURR_GW = DATA['next-event'] - 1
CURR_GW = 38

BASE_PATH = '../'

In [4]:
vpc = calc_vpc(BASE_PATH, SEASON, CURR_GW)
vpc.head()

,value,name,position,display_name,bonus,bonus_weighted,bps,bps_weighted,total_points,total_points_weighted,vpc_ratio,vpc_ratio_weighted
0,5.0,Aaron Cresswell 402,Defender,Aaron Cresswell,0.000000,0.000000,6.763158,0.191215,0.894737,0.025451,0.178947,0.005090
1,4.9,Aaron Lennon 83,Midfielder,Aaron Lennon,0.052632,0.001595,3.500000,0.102867,1.078947,0.031604,0.220193,0.006450
2,5.0,Aaron Mooy 199,Midfielder,Aaron Mooy,0.131579,0.003616,10.815789,0.310198,2.000000,0.057196,0.400000,0.011439
3,7.1,Aaron Ramsey 14,Midfielder,Aaron Ramsey,0.105263,0.002983,7.842105,0.227044,2.394737,0.069813,0.337287,0.009833
4,4.5,Aaron Rowe 607,Midfielder,Aaron Rowe,0.000000,0.000000,-0.222222,-0.006073,0.222222,0.005773,0.049383,0.001283


In [70]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

def get_trace(df, position):
    return go.Scatter(
        x = df['value'],        
        y = df['total_points'],
        name= (position+'s'),
        text = df['display_name'],
        mode = 'markers',        
        marker=dict(color = map_position_to_color(position),
                    size = df['vpc_ratio'], 
                    sizeref = 0.001, 
                    sizemode = 'area'),
        hovertemplate = "<b>%{text}</b><br><br>" +
            "Value: %{y:.2f}</br>"+
            "Cost: %{x:.2f}£</br>"+
            "<extra></extra>")

goalkeepers = vpc[(vpc['position']=='Goalkeeper') & (vpc['total_points']>0.1)]
defenders = vpc[(vpc['position']=='Defender') & (vpc['total_points']>0.1)]
midfielders = vpc[(vpc['position']=='Midfielder') & (vpc['total_points']>0.1)]
forwards = vpc[(vpc['position']=='Forward') & (vpc['total_points']>0.1)]

trace_gkp = get_trace(goalkeepers,'Goalkeeper')
trace_def = get_trace(defenders,'Defender')
trace_mid = get_trace(midfielders,'Midfielder')
trace_fwd = get_trace(forwards,'Forward')

data = [trace_gkp,trace_def,trace_mid,trace_fwd]

updatemenus = list([
    dict(active=0,
         pad = {'r': 0, 't': 10},
         x = 0,
         y = 1.18,
         type = 'buttons',
         bgcolor = 'rgba(255,255,255,100)',
         direction = 'right',
         xanchor = 'left',
         buttons=list([   
            dict(label = 'All',
                 method = 'update',
                 args = [{'visible': [True, True, True, True]}]),
            dict(label = 'Goalkeepers',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]}]),
            dict(label = 'Defenders',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]}]),
            dict(label = 'Forwards',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]}])
        ]),
    )
])

layout = go.Layout(
    modebar={'bgcolor': 'rgba(0,0,0,0)'},
    hovermode = 'closest',
    showlegend=False,
    updatemenus=updatemenus, 
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=go.layout.XAxis(
        showgrid=False,
        zeroline=False,
        color='white',
        showticklabels=False,
        title=go.layout.xaxis.Title(
            text='Cost',
            font=dict(
                size=18,
                color='white'
            )
        )
    ),
    yaxis=go.layout.YAxis(  
        showgrid=False,
        zeroline=False,
        color='white',
        showticklabels=False,
        title=go.layout.yaxis.Title(
            text='Value',
            font=dict(
                size=18,
                color='white'
            )
        )
    )
)
fig = go.Figure(data=data, layout=layout)

In [71]:
#plotly.offline.iplot(fig)
plotly.plotly.plot(fig,filename="vpc")

'https://plot.ly/~antoniaelek/164'